# Create a Plan Position Indicator (PPI) Plot
A Plan Position Indicator (PPI) plot is a common plot requested by radar scientists. Let's show how to create this plot using `xradar`

## Imports

In [ ]:
import os
import xarray as xr
import xradar as xd
from urllib.parse import urljoin
from urllib.request import urlretrieve

## Read in some data
Fetching CfRadial1 radar data file from wradlib-data repository.

In [ ]:
def fetch_cfradial1_file():
    fname = "cfradial1_data.nc"
    if not os.path.exists(fname):
        base_url = "https://raw.githubusercontent.com/wradlib/wradlib-data/main/netcdf/"
        filename = "cfrad.20080604_002217_000_SPOL_v36_SUR.nc"
        url = urljoin(base_url, filename)
        urlretrieve(url, filename=fname)
    return fname


filename = fetch_cfradial1_file()

Read the data using the `cfradial1` engine

In [ ]:
ds = xr.open_dataset(filename, group="sweep_0", engine="cfradial1")
display(ds)

## Add georeferencing
Notice how after applying georeferencing, using the `get_x_y_z` function, we have x, y, and z in our dataset!

In [ ]:
ds = xd.georeference.get_x_y_z(ds)
display(ds)

## Plot our Data
Now, let's create our PPI plot!

In [ ]:
ds.DBZ.plot(x='x', y='y', cmap='inferno')